In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import argparse
import pickle as pkl
import time
from copy import deepcopy
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm

In [2]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from random import shuffle
import torch.nn.functional as F
from torchvision.transforms import v2
from torch import nn

c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, end_maxpool = False):
        super(ResidualBlock, self).__init__()
        if(downsample is not None):
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU(inplace=False),
                            nn.MaxPool2d(kernel_size=2, stride=2)
                            )  # Changed inplace to False
        else:
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.Hardtanh(min_val=-1.0, max_val=1.0, inplace=False)
                            )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(out_channels),
                        nn.Hardtanh(min_val=-1.0, max_val=1.0, inplace=False))  # Changed inplace to False
        self.downsample = downsample
        self.relu = nn.Hardtanh(min_val=-1.0, max_val=1.0, inplace=False)  # Changed inplace to False
        self.out_channels = out_channels
        self.end_maxpool = end_maxpool

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out = out + residual
        if self.end_maxpool:
            out = F.relu(out, inplace=False)
        else:
            out = F.hardtanh(out, inplace=False, min_val=-1.0, max_val=1.0)   # Use non-in-place ReLU
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2, in_chanels = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_chanels, 64, kernel_size = 7, stride = 1, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace=False))
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2, end_maxpool = True)
        self.avgpool = nn.MaxPool2d(7, stride=1)
        self.fc = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1, end_maxpool = False):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=1, padding='same'),
                nn.BatchNorm2d(planes),
                nn.ReLU(inplace=False),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            if i == blocks-1 and end_maxpool:
                layers.append(block(self.inplanes, planes, end_maxpool = True))
            else:
                layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.hardtanh(x)
        x = self.fc2(x)
        return x

In [4]:
model_resnet = ResNet(ResidualBlock, [5, 6, 6, 4], num_classes = 101, in_chanels=6)

In [5]:
model_resnet=torch.load("best_resnet_nCaltech101_Hardtanh_ReLUmaxpool_EST_FC2_corrected_exp.pt", weights_only=False)
model_resnet.eval().to("cuda")
print(model_resnet)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer0): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Hardtanh(min_val=-1.0, max_val=1.0)
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Hardtanh(min_val=-1.0, max_val=1.0)
      )
      (relu): Hardtanh(min_val=-1.0, max_val=1.0)
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kerne

In [6]:
temp = torch.rand((1,6,224,224))
model_resnet(temp.to("cuda"))

tensor([[ 5.4523,  2.3179,  2.4930,  0.5591,  1.1282, -1.4592,  1.9931, -0.3587,
          3.3126, -2.9173, -0.2203, -0.3653, -5.7149,  1.3805,  1.7708, -0.2346,
         -2.8358,  2.6006,  8.1917, -3.3833, -3.3626, -1.2714, -6.8086, -0.4964,
         -1.7192, -3.0392, -0.6481,  1.1104, -3.0696, -2.1792,  4.4010,  1.1994,
          5.2197, -3.1094, -5.6506,  3.6647, -2.8838, -2.4429,  1.1217, -4.0740,
          0.9619, -1.6994, -3.1482,  3.3794,  1.3600,  2.0743,  5.3976, -4.1290,
         -0.4036, -1.3789,  0.9460, -6.6034, -5.5881, -1.3458, -2.1289,  3.9662,
         -4.7638, -5.3037, -1.5208, -1.0767,  7.7006,  2.9608, -0.4339,  0.7386,
          5.2189, -1.5029,  3.4371,  2.6185, -1.4091,  5.1200, -0.1363,  2.5581,
         -3.8921, -2.2180,  3.8134,  3.9212, -1.0427, -0.1211, -2.8506,  0.5521,
         -0.2868, -0.9606, -4.0665, -0.6620,  3.5067,  8.7958, -0.6043, -4.4645,
          1.0025, -1.3193, -3.8457, -6.7153,  2.4497,  4.1523,  1.1178, -3.2993,
         -0.4880, -0.3057, -

In [7]:
from SNN.Check_Hardtanh import fuse_conv_and_bn
from SNN.Check_Hardtanh import SpikingConv2D
from SNN.Check_Hardtanh import MaxMinPool2D
from SNN.Check_Hardtanh import LayerSNN_all
from SNN.Check_Hardtanh import SpikingDense_positive_tanH, SpikingDense_all_all, SpikingDense

In [8]:
class ResNet_ttfs(nn.Module):
    def __init__(self, model : ResNet):
        super(ResNet_ttfs, self).__init__()
        model.eval()
        robustness_params={
            'noise':0.0,
            'time_bits':0,
            'weight_bits': 0,
            'latency_quantiles':0.0
        }
        model.to('cuda')
        conv_fused = fuse_conv_and_bn(model.conv1[0], model.conv1[1], device = 'cuda')
        self.conv_first = SpikingConv2D(64, "temp1", device = 'cuda', padding=(3,3), stride=2, kernel_size=(7,7),robustness_params=robustness_params, kernels=conv_fused.weight.data, biases= conv_fused.bias.data)
        max_vect = torch.tensor([1]*6)
        tmin, tmax, max_vect, scalar = self.conv_first.set_params(0.0,1.0,max_vect)
        self.tmin_post_pool = tmax
        self.pool = MaxMinPool2D(3, tmax.data,2,1).to("cuda")
        self.layer0SNN = LayerSNN_all(model.layer0, 64, 64, 5,device = 'cuda')
        tmax_prev = tmax
        tmin, tmax, max_vect, scalar = self.layer0SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer1SNN = LayerSNN_all(model.layer1, 64, 128, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer1SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer2SNN = LayerSNN_all(model.layer2, 128, 256, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer2SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer3SNN = LayerSNN_all(model.layer3, 256, 512, 4,device = 'cuda',end_maxpool=True)
        tmin, tmax, max_vect, scalar = self.layer3SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.pool2 = MaxMinPool2D(7, tmax.data,1,0).to("cuda")
        # temp = torch.ones((512,2,2))
        # max_vect = ((temp.T)*max_vect[:512]).T
        # max_vect = max_vect.view(-1)
        print(max_vect.shape)
        max_vect = (torch.ones([512, 1, 1]).to("cuda").T*max_vect[:512]).T.contiguous().view(512).to("cuda")

        weights = model.fc.weight.detach().clone()
        biases = model.fc.bias.detach().clone()
        self.layer_fc = SpikingDense_positive_tanH(256, 512,"test",robustness_params=robustness_params,device = 'cuda',weights=weights, biases=biases)
        
        tmin, tmax, max_vect, scalar = self.layer_fc.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer_fc2 = SpikingDense_all_all(101,256, '',model.fc2.weight, model.fc2.bias,robustness_params=robustness_params,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer_fc2.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.tmin, self.tmax = tmin, tmax
        self.scalar = scalar
    
    def forward(self, tj):
        spike_sum, v_sqare_sum, not_spiked_sum = 0,0.0, 0

        x, spike, v, not_spiked  = self.conv_first(tj)
        spike_sum += spike
        v_sqare_sum +=v
        not_spiked_sum += not_spiked
        x, spike = self.pool(x)

        spike_sum += spike
        # print(x.shape)
        x = x
        x, spike1, v1, not_spiked1 = self.layer0SNN(torch.concat((x, torch.ones(x.shape).to("cuda") * self.tmin_post_pool),dim=1))
        x, spike2, v2, not_spiked2 = self.layer1SNN(x)

        x, spike3, v3, not_spiked3 = self.layer2SNN(x)

        x, spike4, v4, not_spiked4 = self.layer3SNN(x)
        x = x[:512]
        # print(x[1].max(),x[1].min())
        x, spike5 = self.pool2(x)
        x = x.contiguous().view(x.size(0), -1)

        x, spike6, v5, not_spiked5 = self.layer_fc(x)

        x, spike7, v6, not_spiked6 = self.layer_fc2(x)

        spike_sum += spike1+spike2+spike3+spike4+spike5+spike6+spike7
        v_sqare_sum += v1+v2+v3+v4+v5+v6
        not_spiked_sum += not_spiked1+not_spiked2+not_spiked3+not_spiked4+not_spiked5+not_spiked6
        return x, spike_sum, v_sqare_sum, not_spiked_sum

In [9]:
model_ttfs = ResNet_ttfs(model_resnet)

c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(t_min + self.B_n*max_V+eps_V, dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:312: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(max(t_min + self.B_n*max_V+eps_V, minimal_t_max), dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:281: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_min = torch.tensor

torch.Size([1024])


C:\Users\nikos\AppData\Local\Temp\ipykernel_28764\3812665932.py:32: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3575.)
  max_vect = (torch.ones([512, 1, 1]).to("cuda").T*max_vect[:512]).T.contiguous().view(512).to("cuda")
c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_input = torch.concat((torch.tensor(in_ranges_max).to(self.device), torch.tensor([(1)]).to(self.device)))
c:\D\time_to_spike\in_torch\SNN\Check_Hardta

In [10]:
temp = torch.rand((1,6,224,224))
temp_ttfs = 1 - temp
# print(model_ttfs.forward(temp_ttfs))
out, spike_sum, v_sqare_sum, not_spike = model_ttfs.forward(temp_ttfs.to("cuda"))

print(spike_sum, v_sqare_sum, not_spike)
print((model_ttfs.tmax - out)*model_ttfs.scalar)

tensor(13565625, device='cuda:0') tensor(5038962., device='cuda:0', grad_fn=<AddBackward0>) tensor(27457837, device='cuda:0')
tensor([[5.3928, 2.3536, 2.5330, 0.8869, 1.1957, 0.0000, 1.7190, 0.0000, 3.3793,
         0.0000, 0.0000, 0.0000, 0.0000, 1.5237, 1.8430, 0.0000, 0.0000, 2.6268,
         8.1474, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.1469, 0.0000, 0.0000, 4.1434, 1.2764, 5.3309, 0.0000, 0.0000, 3.4391,
         0.0000, 0.0000, 1.0816, 0.0000, 0.9725, 0.0000, 0.0000, 3.0529, 1.1003,
         2.0662, 5.5486, 0.0000, 0.0000, 0.0000, 1.0284, 0.0000, 0.0000, 0.0000,
         0.0000, 3.9251, 0.0000, 0.0000, 0.0000, 0.0000, 7.8858, 3.0595, 0.0000,
         0.8173, 4.9288, 0.0000, 3.2169, 2.8061, 0.0000, 5.0966, 0.0911, 2.6196,
         0.0000, 0.0000, 3.6047, 3.8367, 0.0000, 0.1377, 0.0000, 0.6752, 0.0000,
         0.0000, 0.0000, 0.0000, 3.6294, 8.5780, 0.0000, 0.0000, 1.0805, 0.0000,
         0.0000, 0.0000, 2.1726, 3.9640, 1.0372, 0.0000, 0.0000,

In [11]:
model_resnet(temp.to("cuda"))

tensor([[ 5.3985,  2.3491,  2.5349,  0.8885,  1.1952, -1.6798,  1.7158, -0.5487,
          3.3793, -3.2467, -0.3426, -0.3958, -5.7061,  1.5273,  1.8409, -0.1372,
         -2.9144,  2.6233,  8.1514, -3.3649, -3.0060, -1.3111, -6.9109, -0.4449,
         -1.5781, -2.9028, -0.7408,  1.1453, -2.6868, -2.2820,  4.1407,  1.2776,
          5.3277, -3.1857, -5.8135,  3.4380, -2.7007, -2.8015,  1.0776, -4.1426,
          0.9698, -1.7744, -3.0817,  3.0507,  1.0978,  2.0646,  5.5488, -4.5535,
         -0.2897, -1.3255,  1.0288, -6.3236, -5.6044, -1.4606, -2.3727,  3.9237,
         -4.6522, -5.1915, -1.5874, -1.2165,  7.8829,  3.0603, -0.3881,  0.8178,
          4.9319, -1.4403,  3.2193,  2.8049, -1.6862,  5.1011,  0.0924,  2.6192,
         -4.0046, -2.4191,  3.6043,  3.8306, -1.1436,  0.1393, -2.7476,  0.6722,
         -0.2687, -0.6752, -3.9948, -0.3971,  3.6331,  8.5768, -0.6064, -4.5187,
          1.0824, -1.4147, -3.9462, -6.3654,  2.1742,  3.9611,  1.0387, -3.2545,
         -0.4068, -0.2905, -

In [12]:
from torchvision.transforms import v2
torch.manual_seed(19)

transforms = v2.Compose([
    # v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(60),
    v2.ToDtype(torch.float32)
])


class NCaltech101ImageDataset(Dataset):
    def __init__(self, img_dir_file, transform=None, target_transform=None):
        self.images = np.load(img_dir_file + '_x.npy')
        self.labels = np.load(img_dir_file + '_y.npy')
        self.transform = transform
        self.transform = transform
        self.stage = 0

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        label_temp = np.zeros((101,))
        label_temp[label] = 1
        if self.stage == 0:
            return self.transform(torch.tensor(image)), torch.tensor(label_temp)
        else:
            return torch.tensor(image), torch.tensor(label_temp)
    
    def set_stage(self, stage):
        self.stage = stage

In [13]:
from torch.utils.data import DataLoader
# training_data = NCarsImageDataset("./Datasety/nCars_train_EST_exp_", transform=transforms)
data = NCaltech101ImageDataset("./Datasety/Ncaltech101_EST_exp_corr", transform=transforms)


In [16]:
data.set_stage(1)
spike_acc = 0
not_spike_acc = 0
v_sqare_acc = 0.0
MAE_sum = 0
MSE_sum = 0
TP = 0
for i in tqdm(range(len(data))):#10
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            temp_data, temp_label = deepcopy(data[i])
            temp_ttfs = (1 - temp_data).unsqueeze(0).to("cuda")
            temp_data = temp_data.to("cuda")
            # print(temp_data.shape)
            # print(model_ttfs.forward(temp_ttfs))
            out, spike_sum, v_sqare_sum, not_spike = model_ttfs.forward(temp_ttfs)
            spike_acc+=spike_sum
            not_spike_acc += not_spike
            v_sqare_acc+=v_sqare_sum
            output = ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:]
            output_true = model_resnet(temp_data.unsqueeze(0))
            MAE = torch.sum((output-output_true).abs())/101.0
            MSE = torch.sqrt(torch.sum((output-output_true)*(output-output_true)))/101.0
            MAE_sum += MAE
            MSE_sum += MSE
            TP += (torch.argmax(temp_label)==torch.argmax(output))
            del temp_data, temp_ttfs
    if i%100==0:
        torch.cuda.empty_cache()
print(spike_acc/len(data))
print("Mean MAE:", MAE_sum/len(data))
print("Mean MSE:", MSE_sum/(len(data)))
print("accuracy:",float(TP)/len(data))
print("relative fraction: ", v_sqare_acc/not_spike_acc)
print("Sum V_square^2/treshold^2: ", v_sqare_acc/len(data))
print("Not spiked: ", not_spike/len(data))

100%|██████████| 8709/8709 [3:03:00<00:00,  1.26s/it]  

tensor(13535074., device='cuda:0')
Mean MAE: tensor(0.0072, device='cuda:0')
Mean MSE: tensor(0.0009, device='cuda:0')
accuracy: 0.6795269261683317
relative fraction:  tensor(0.1827, device='cuda:0')
Sum V_square^2/treshold^2:  tensor(5021309.5000, device='cuda:0')
Not spiked:  tensor(3158.1245, device='cuda:0')


In [17]:
print("Not spiked: ", not_spike_acc/len(data))

Not spiked:  tensor(27490362., device='cuda:0')
